In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
idx = pd.IndexSlice
import statsmodels.api as sm

In [2]:
from statsmodels.regression.linear_model  import  OLS
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint
from sklearn.model_selection import train_test_split

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [5]:
#прочитать csv матаданные
nifty_meta = pd.read_csv('data/nifty_meta.csv')
nifty_meta.head(4)

,Company Name,Industry,Symbol,Series,ISIN Code
0,ACC Ltd.,CEMENT & CEMENT PRODUCTS,ACC,EQ,INE012A01025
1,Adani Ports and Special Economic Zone Ltd.,SERVICES,ADANIPORTS,EQ,INE742F01042
2,Adani Transmission Ltd.,POWER,ADANITRANS,EQ,INE931S01010
3,Ambuja Cements Ltd.,CEMENT & CEMENT PRODUCTS,AMBUJACEM,EQ,INE079A01024


In [7]:
#получить список тикеров с отраслью, равной FINANCIAL SERVICES
tickers = list(nifty_meta[nifty_meta.Industry == 'FINANCIAL SERVICES'].Symbol)
print(tickers)
print(len(tickers))

['AXISBANK', 'BAJFINANCE', 'BAJAJFINSV', 'BAJAJHLDNG', 'BANDHANBNK', 'BANKBARODA', 'GICRE', 'HDFCAMC', 'HDFCBANK', 'HDFCLIFE', 'HDFC', 'ICICIBANK', 'ICICIGI', 'ICICIPRULI', 'IBULHSGFIN', 'INDUSINDBK', 'KOTAKBANK', 'L&TFH', 'PEL', 'PFC', 'PNB', 'SBILIFE', 'SRTRANSFIN', 'SBIN', 'NIACL']
25


In [8]:
# даты начала и окончания тестирования
fromdate = datetime.datetime(2010, 1, 1)
todate =datetime.datetime(2020, 6, 15)

In [10]:
# прочитать данные о ценах
prices = pd.read_csv('data/prices.csv',index_col=['ticker', 'date'], parse_dates=True)
prices.head(5)

open        high         low       close
ticker   date                                                      
AXISBANK 2010-01-04  198.779999  199.990005  197.619995  140.712021
         2010-01-05  199.600006  204.000000  199.139999  143.647934
         2010-01-06  204.000000  205.740005  198.509995  141.236801
         2010-01-07  199.289993  201.199997  197.820007  142.236725
         2010-01-08  200.600006  203.979996  199.270004  144.094727

In [11]:
# удалить тикеры, если у нас есть данные менее чем за 10 лет.
min_obs = 2520
nobs = prices.groupby(level='ticker').size()
keep = nobs[nobs>min_obs].index

In [12]:
prices = prices.loc[idx[keep,:], :]
prices.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 38580 entries, ('AXISBANK', Timestamp('2010-01-04 00:00:00')) to ('SRTRANSFIN', Timestamp('2020-06-15 00:00:00'))
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    38579 non-null  float64
 1   high    38579 non-null  float64
 2   low     38579 non-null  float64
 3   close   38579 non-null  float64
dtypes: float64(4)
memory usage: 1.4+ MB
